In [1]:
%load_ext cython

# Standard Template Library Support!

We're so excited to try it out! Let's go!

In [2]:
%%cython
# cython: language_level = 3

from libcpp.vector cimport vector

cdef vector[int] vect
cdef int i
for i in range(10):
    vect.push_back(i)
for i in range(10):
    print(vect[i], end=',')

CompileError: command '/usr/bin/clang' failed with exit status 1

## Oops, you need to change the (export) language to C++...

In [3]:
%%cython -a
# cython: language_level = 3
# distutils: language = c++

from libcpp.vector cimport vector

cdef vector[int] vect
cdef int i
for i in range(10):
    vect.push_back(i)
for i in range(10):
    print(vect[i], end=',')

0,1,2,3,4,5,6,7,8,9,

# Let's have a closer look

In [4]:
%%cython
# cython: language_level = 3
# distutils: language = c++

from libcpp.vector cimport vector

# Make a vector

cdef vector[int] vect
cdef int i, n = 10000000
for i in range(n):
    vect.push_back(i)    
        
# Iteration: easiest      
        
def method1():
    cdef int x, a = 0
    for x in vect:
        a += x % 5
    return a
    
# Iteration with range: easy    
    
def method2():
    cdef int i, a = 0, n = vect.size()
    for i in range(n):
        a += vect[i] % 5
    return a    
    
# Iteration: hard
    
from cython.operator cimport dereference, preincrement

def method3():
    cdef int x, a = 0
    cdef vector[int].iterator itr
    itr = vect.begin()
    while itr != vect.end():
        a += dereference(itr) % 5
        preincrement(itr)
    return a

In [5]:
print(method1())
print(method2())
print(method3())

20000000
20000000
20000000


In [6]:
%timeit method1()
%timeit method2()
%timeit method3()

10 loops, best of 3: 18.9 ms per loop
10 loops, best of 3: 20 ms per loop
10 loops, best of 3: 19.5 ms per loop


# Auto-converting TO Python objects

In [10]:
%%cython
# cython: language_level = 3
# distutils: language = c++

from libcpp.vector cimport vector
from libcpp.list cimport list
from libcpp.map cimport map
from libcpp.set cimport set
from libcpp.pair cimport pair
from libcpp.deque cimport deque
from libcpp.queue cimport queue

def try_vector():
    cdef vector[int] x
    cdef int i, n = 5
    for i in range(n):
        x.push_back(i)
    cdef object o = x
    print(type(o))
    
def try_list():
    cdef list[int] x
    cdef int i, n = 5
    for i in range(n):
        x.push_back(i)
    cdef object o = x
    print(type(o))
    
def try_map():
    cdef map[int, int] x
    cdef int i, n = 5
    for i in range(n):
        x.insert([i, i])
    cdef object o = x
    print(type(o))
    
def try_set():
    cdef set[int] x
    cdef int i, n = 5
    for i in range(n):
        x.insert(i)
    cdef object o = x
    print(type(o))    
    
def try_pair():
    cdef pair[int, int] x
    x.first = 1
    x.second = 2
    cdef object o = x
    print(type(o))  
    
# def try_deque():
#     cdef deque[int] x
#     cdef int i, n = 5
#     for i in range(n):
#         x.push_front(i)
#     cdef object o = x
#     print(type(o))  
    
# def try_queue():
#     cdef queue[int] x
#     cdef int i, n = 5
#     for i in range(n):
#         x.push(i)
#     cdef object o = x
#     print(type(o))  

In [11]:
try_vector()
try_list()
try_map()
try_set()
try_pair()

<class 'list'>
<class 'list'>
<class 'dict'>
<class 'set'>
<class 'tuple'>


# Auto-converting FROM Python objects

"You want to erase debt!?"  

"In THIS economy!?"

In [12]:
%%cython
# cython: language_level = 3
# distutils: language = c++

from libcpp.map cimport map
from libcpp.string cimport string

def erase_debt_but_only_for_me(map[string, double] debtors):
    debtors[b'Caleb'] = 0.0

In [13]:
# Make a dict of debtors
debtors = {
    b'John': 100.0,
    b'Caleb': 5000.0,
    b'Sarah': 250.0
    }

erase_debt_but_only_for_me(debtors)
print(debtors)

{b'John': 100.0, b'Caleb': 5000.0, b'Sarah': 250.0}


## Must return a modified dictionary

In [14]:
%%cython
# cython: language_level = 3
# distutils: language = c++
from libcpp.map cimport map
from libcpp.pair cimport pair
from libcpp.string cimport string

def erase_debt_but_only_for_me(map[string, double] debtors):
    debtors[b'Caleb'] = 0.0
    return debtors

In [15]:
new_debtors = erase_debt_but_only_for_me(debtors)
print(new_debtors)

{b'John': 100.0, b'Caleb': 0.0, b'Sarah': 250.0}


In [16]:
print(debtors)

{b'John': 100.0, b'Caleb': 5000.0, b'Sarah': 250.0}
